In [6]:
import rasterio
import numpy as np
import pandas as pd
import cv2
from catboost import CatBoostClassifier

from Nesterov_koef import CFD
from api import call_api

In [10]:
model = CatBoostClassifier()
model.load_model('fire_falcon')

In [16]:
def transform(img_path):
    with rasterio.open(img_path) as src:
        red = src.read(1)  # B02 - Blue
        green = src.read(2)  # B03 - Green
        blue = src.read(3)  # B04 - Red
        ik = src.read(4)

        cords = get_coords(img_path)
        json = call_api(cords[0], cords[1], get_time(img_path)) 
        data = pd.DataFrame(json)
        data = data[['max_temp', 'dewpt', 'precip']].to_numpy() #'precip', 'dewpt', 'max_temp'
        coefs = CFD(data[:, 0], data[:, 1], data[:, 2])
        vector_expanded = np.tile(coefs, (135*106, 1))

        img = np.stack([red, green, blue, ik], axis=-1)

        img = cv2.resize(img, (135, 106))
        
        img = img.reshape([135 * 106, 4])
        result = np.hstack((img, vector_expanded))

        return result
    
def get_coords(img_path):
    with rasterio.open(img_path) as src:
        return [src.bounds.top, src.bounds.left]
    
def get_time(img_path):
     return img_path[4:-5]

def get_size(img_path):
    with rasterio.open(img_path) as src:
        return [src.width, src.height]

def get_prediction(tiff_path):
    image = transform(tiff_path)
    predict = model.predict(image)
    predict = predict.reshape([106,135,1])

    size = get_size(tiff_path)
    #return_image = cv2.resize(predict, size)
    return predict#return_image

In [18]:
img = get_prediction("08//2021-06-08.tiff")

In [19]:
img.shape

(106, 135, 1)